In [2]:
"""Set up connection to Google Drive."""

# import Drive helper and mount drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Suppress output of this cell
%%capture

"""Install and import dependencies."""

# Install and import spotipy to access Spotify API
!pip install spotipy
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 

# Import configparser for loading configuration file to call Spotify API
import configparser

# Import pandas for dataframes
import pandas as pd 

# Import matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline 

# Import numpy to help with plots
import numpy as np

# Import scikit-learn for scaling and k-means clustering
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans

In [0]:
"""Define path variables."""

# Main path in Google Drive
homePath = '/content/drive/My Drive/Colab Notebooks'

# Path to the Spotify configuration file
configFilePath = f'{homePath}/bin/spotify_client_config.txt'

# Path to save data
dataPath = f'{homePath}/Data/Spotify audio features'

In [8]:
def init_spotify_client(configFilePath: str):
  """
  Initialise Spotify client to use Spotify API.
  
  :param str configFilePath: Path to the Spotify config file
  :returns: Spotify client
  """
  # Load the Spotify client configuration file
  config = configparser.ConfigParser()
  config.read(configFilePath)
  
  print(f'Loaded config file with settings for: {config.sections()}')

  # Use configuration to set up Spotify client authorisation
  client_credentials_manager = \
  SpotifyClientCredentials(client_id=config['developer.spotify.com']['cid'], \
                           client_secret=config['developer.spotify.com']['secret'])

  # Initialise the Spotify client
  sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
  
  print('Authorised Spotify client has been initialised')
  
  return sp


# Initialise the Spotify client
sp = init_spotify_client(configFilePath)

Loaded config file with settings for: ['developer.spotify.com']
Authorised Spotify client has been initialised


In [17]:
def get_artist_albums(sp, artist_url: str):
  """
  Get ths IDs and names of all albums for an artist on Spotify, and returns them in a list.
  
  :param sp: Spotify client
  :param str artist_url: The Spotify URL of artist
  :returns: List containing the IDs and names of albums for the artist
  """
  
  # Set limit to 50, which is the maximum
  request_limit = 50
  
  # Get all album information from Spotify
  artist_album_info = sp.artist_albums(artist_url, album_type='album', limit=request_limit)
  artist_single_info = sp.artist_albums(artist_url, album_type='single', limit=request_limit)
  artist_compilation_info = sp.artist_albums(artist_url, album_type='compilation', limit=request_limit)
  
  """
  albums = artist_album_info['items']
  while artist_album_info['next']:
    artist_album_info = spotify.next(artist_album_info)
    albums.extend(artist_album_info['items'])
  """
  
  return artist_album_info
  
# Oku Hanako's Spotify URL
oku_hanako_url = 'https://open.spotify.com/artist/2tOwqfTtAMswbLySSaTRYR'


test = get_artist_albums(sp, oku_hanako_url)
print(test)

{'href': 'https://api.spotify.com/v1/artists/2tOwqfTtAMswbLySSaTRYR/albums?offset=0&limit=50&include_groups=album', 'items': [{'album_group': 'album', 'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2tOwqfTtAMswbLySSaTRYR'}, 'href': 'https://api.spotify.com/v1/artists/2tOwqfTtAMswbLySSaTRYR', 'id': '2tOwqfTtAMswbLySSaTRYR', 'name': 'Hanako Oku', 'type': 'artist', 'uri': 'spotify:artist:2tOwqfTtAMswbLySSaTRYR'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'US', 'UY', 'VN', 'ZA'], 'external_urls': {'spotify': 'https://open.

In [0]:
def get_album_tracks(sp, album_url: str):
  """
  Get IDs and names of all tracks in the album, and returns them in a list.
  
  :param sp: Spotify client
  :param str album_url: URL of album on Spotify
  :returns: List containing IDs and names of tracks in the album
  """
  
  # Get album information from Spotify
  album = sp.album(album_url)
  num_tracks = album['total_tracks']
  album_name = album['name']
  
  print(f'Found {num_tracks} in {album_name}')
  
  # Initialise empty list
  track_list = []
  
  print(f'Start extracting track IDs and names from {album_name}')

  # Iterate over all tracks
  for track_idx in range(0, num_tracks):
    # Record the track ID and name
    track_list.append([
        album['tracks']['items'][track_idx]['id'],
        album['tracks']['items'][track_idx]['name']
    ])

  print(f'Finished extracting track IDs and names from {album_name}')
    
  return track_list


